# Sequential Model Training with Memory Optimization
This notebook trains models one by one with caching for efficient memory usage on Kaggle P100 GPU.

**Instructions:** Run cells sequentially. Skip any model cell if you don't want to train that specific model.

In [ ]:
!pip install -U 'tensorflow[and-cuda]'

In [ ]:
# Import Required Libraries
import tensorflow as tf
from tensorflow.keras.applications import VGG16, VGG19, InceptionV3, Xception, ResNet50, DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import os
import numpy as np
import pandas as pd
import gc
from tensorflow.keras import backend as K

# Enable mixed precision for faster training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

# Set memory growth to avoid OOM errors
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU Available: {len(gpus)} GPU(s)")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found")

print("TensorFlow version:", tf.__version__)

In [ ]:
# Download dataset (for Kaggle, use the input path directly)
# Uncomment the following for local development:
# import kagglehub
# path = kagglehub.dataset_download("vipoooool/new-plant-diseases-dataset")
# print("Path to dataset files:", path)

# For Kaggle kernel:
train = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
valid = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'

# For local development:
# train = "C:\\Users\\junu\\.cache\\kagglehub\\datasets\\vipoooool\\new-plant-diseases-dataset\\versions\\2\\New Plant Diseases Dataset(Augmented)\\New Plant Diseases Dataset(Augmented)\\train"
# valid = "C:\\Users\\junu\\.cache\\kagglehub\\datasets\\vipoooool\\new-plant-diseases-dataset\\versions\\2\\New Plant Diseases Dataset(Augmented)\\New Plant Diseases Dataset(Augmented)\\valid"

In [ ]:
# Configuration
image_size = (128, 128)  # Change to (224, 224) or (299, 299) for better accuracy
batch_size = 64          # Reduce to 32 if using larger image sizes
CACHE_DIR = '/kaggle/working/cache'  # Cache directory for preprocessed data
os.makedirs(CACHE_DIR, exist_ok=True)

print(f"Image size: {image_size}")
print(f"Batch size: {batch_size}")
print(f"Cache directory: {CACHE_DIR}")

In [ ]:
# Create datasets with caching for memory efficiency
def create_cached_datasets():
    """
    Create training and validation datasets with caching.
    This prevents reloading data from disk for each model.
    """
    # Training dataset
    train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        train,
        seed=123,
        image_size=image_size,
        batch_size=batch_size,
        label_mode='categorical',
        shuffle=True
    )
    
    # Validation dataset
    val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        valid,
        seed=123,
        image_size=image_size,
        batch_size=batch_size,
        label_mode='categorical',
        shuffle=False
    )
    
    # Save class names
    class_names = train_dataset.class_names
    num_classes = len(class_names)
    
    print(f"Number of classes: {num_classes}")
    
    # Data augmentation for training
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.Rescaling(1./255),
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomFlip("vertical"),
        tf.keras.layers.RandomRotation(0.2),
        tf.keras.layers.RandomZoom(0.2),
        tf.keras.layers.RandomContrast(0.2),
    ])
    
    # Normalization for validation
    normalization = tf.keras.layers.Rescaling(1./255)
    
    # Apply transformations and cache
    train_dataset = train_dataset.map(
        lambda x, y: (data_augmentation(x, training=True), y),
        num_parallel_calls=tf.data.AUTOTUNE
    ).cache(os.path.join(CACHE_DIR, 'train_cache')).prefetch(tf.data.AUTOTUNE)
    
    val_dataset = val_dataset.map(
        lambda x, y: (normalization(x), y),
        num_parallel_calls=tf.data.AUTOTUNE
    ).cache(os.path.join(CACHE_DIR, 'val_cache')).prefetch(tf.data.AUTOTUNE)
    
    return train_dataset, val_dataset, class_names, num_classes

# Create datasets once
print("Creating and caching datasets...")
train_dataset, val_dataset, class_names, num_classes = create_cached_datasets()
print("✓ Datasets created and cached successfully!")

In [ ]:
# Model creation function
def create_model(base_model_class, model_name, num_classes):
    """
    Create a model with the specified base architecture.
    """
    # Create base model
    base_model = base_model_class(
        weights='imagenet',
        include_top=False,
        input_shape=image_size + (3,)
    )
    
    # Freeze base model initially
    base_model.trainable = False
    
    # Build model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    # Dense layers with regularization
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    # Output layer
    predictions = Dense(num_classes, activation='softmax', dtype='float32')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy'],
        jit_compile=True
    )
    
    return model, base_model

# Fine-tuning function
def fine_tune_model(model, base_model, num_layers_to_unfreeze=20):
    """
    Fine-tune the model by unfreezing top layers.
    """
    base_model.trainable = True
    
    # Freeze all layers except the last num_layers_to_unfreeze
    for layer in base_model.layers[:-num_layers_to_unfreeze]:
        layer.trainable = False
    
    # Recompile with lower learning rate
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Callbacks function
def get_callbacks(model_name):
    """
    Create callbacks for training.
    """
    callbacks = [
        EarlyStopping(
            monitor='val_accuracy',
            patience=5,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-7,
            verbose=1
        ),
        ModelCheckpoint(
            filepath=f'{model_name}_best.weights.h5',
            monitor='val_accuracy',
            save_best_only=True,
            save_weights_only=True,
            verbose=1
        )
    ]
    return callbacks

# Memory cleanup function
def clear_memory():
    """
    Clear memory between model trainings to prevent OOM errors.
    """
    K.clear_session()
    gc.collect()
    print("✓ Memory cleared")

# Training function for a single model
def train_single_model(model_name, base_model_class):
    """
    Train a single model with two-stage training.
    """
    print(f"\n{'='*80}")
    print(f"TRAINING {model_name}")
    print(f"{'='*80}")
    
    # Create model
    print(f"\nCreating {model_name} model...")
    model, base_model = create_model(base_model_class, model_name, num_classes)
    print(f"✓ {model_name} model created")
    
    # Stage 1: Initial training
    print(f"\n[STAGE 1] Initial training with frozen base model...")
    initial_epochs = 10
    
    history_stage1 = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=initial_epochs,
        callbacks=get_callbacks(f"{model_name}_stage1")
    )
    
    # Stage 2: Fine-tuning
    print(f"\n[STAGE 2] Fine-tuning with unfrozen top layers...")
    model = fine_tune_model(model, base_model, num_layers_to_unfreeze=20)
    fine_tune_epochs = 10
    
    history_stage2 = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=fine_tune_epochs,
        callbacks=get_callbacks(f"{model_name}_stage2")
    )
    
    # Save model
    model.save(f"{model_name}_plant_disease_model.h5")
    print(f"\n✓ {model_name} model saved")
    
    # Generate predictions
    print(f"\nGenerating predictions for {model_name}...")
    y_pred = model.predict(val_dataset)
    
    # Extract true labels
    y_true = []
    for image_batch, label_batch in val_dataset:
        y_true.append(label_batch)
    y_true = tf.concat(y_true, axis=0).numpy()
    
    # Save predictions
    np.save(f"{model_name}_y_pred.npy", y_pred)
    np.save(f"{model_name}_y_true.npy", y_true)
    print(f"✓ {model_name} predictions saved")
    
    # Calculate accuracy
    y_pred_labels = y_pred.argmax(axis=1)
    y_true_labels = y_true.argmax(axis=1)
    accuracy = np.mean(y_pred_labels == y_true_labels)
    
    print(f"\n{model_name} FINAL VALIDATION ACCURACY: {accuracy*100:.2f}%")
    
    if accuracy >= 0.985:
        print(f"✓ {model_name} achieved >=98.5% accuracy!")
    else:
        print(f"✗ {model_name} did not reach 98.5% accuracy (got {accuracy*100:.2f}%)")
    
    # Clear memory before next model
    del model, base_model
    clear_memory()
    
    return accuracy

print("✓ Helper functions defined")

---
## Model Training Cells
**Run each cell below to train that specific model. Skip any cell to exclude that model from training.**

---

In [ ]:
# Train VGG16
try:
    vgg16_accuracy = train_single_model('VGG16', VGG16)
except Exception as e:
    print(f"❌ Error training VGG16: {str(e)}")
    clear_memory()

In [ ]:
# Train VGG19
try:
    vgg19_accuracy = train_single_model('VGG19', VGG19)
except Exception as e:
    print(f"❌ Error training VGG19: {str(e)}")
    clear_memory()

In [ ]:
# Train InceptionV3
try:
    inceptionv3_accuracy = train_single_model('InceptionV3', InceptionV3)
except Exception as e:
    print(f"❌ Error training InceptionV3: {str(e)}")
    clear_memory()

In [ ]:
# Train Xception
try:
    xception_accuracy = train_single_model('Xception', Xception)
except Exception as e:
    print(f"❌ Error training Xception: {str(e)}")
    clear_memory()

In [ ]:
# Train ResNet50
try:
    resnet50_accuracy = train_single_model('ResNet50', ResNet50)
except Exception as e:
    print(f"❌ Error training ResNet50: {str(e)}")
    clear_memory()

In [ ]:
# Train DenseNet121
try:
    densenet121_accuracy = train_single_model('DenseNet121', DenseNet121)
except Exception as e:
    print(f"❌ Error training DenseNet121: {str(e)}")
    clear_memory()

---
## Results & Comparison
**Run the cells below after training your models to see the comparison.**

---

In [ ]:
# Generate comparison table
from sklearn.metrics import precision_score, recall_score, f1_score

print("="*80)
print("MODEL COMPARISON RESULTS")
print("="*80)

# List of all possible models
all_models = ['VGG16', 'VGG19', 'InceptionV3', 'Xception', 'ResNet50', 'DenseNet121']
results = []

for model_name in all_models:
    try:
        # Load predictions
        y_pred = np.load(f"{model_name}_y_pred.npy")
        y_true = np.load(f"{model_name}_y_true.npy")
        
        # Convert to class labels
        y_pred_labels = y_pred.argmax(axis=1)
        y_true_labels = y_true.argmax(axis=1)
        
        # Calculate metrics
        accuracy = np.mean(y_pred_labels == y_true_labels)
        precision = precision_score(y_true_labels, y_pred_labels, average='weighted')
        recall = recall_score(y_true_labels, y_pred_labels, average='weighted')
        f1 = f1_score(y_true_labels, y_pred_labels, average='weighted')
        
        # Top-5 accuracy
        top5_pred = np.argsort(y_pred, axis=1)[:, -5:]
        top5_accuracy = np.mean([y_true_labels[i] in top5_pred[i] for i in range(len(y_true_labels))])
        
        results.append({
            'Model': model_name,
            'Accuracy (%)': accuracy * 100,
            'Precision (%)': precision * 100,
            'Recall (%)': recall * 100,
            'F1-Score (%)': f1 * 100,
            'Top-5 Accuracy (%)': top5_accuracy * 100,
            'Meets Target (>98.5%)': '✓' if accuracy > 0.985 else '✗'
        })
    except FileNotFoundError:
        print(f"⚠️  {model_name} was not trained (skipped)")
    except Exception as e:
        print(f"Warning: Could not load results for {model_name}: {str(e)}")

if len(results) > 0:
    # Create DataFrame
    comparison_df = pd.DataFrame(results)
    comparison_df = comparison_df.sort_values('Accuracy (%)', ascending=False)
    
    print("\n" + comparison_df.to_string(index=False))
    print("\n" + "="*80)
    
    # Count models above target
    models_above_target = comparison_df[comparison_df['Meets Target (>98.5%)'] == '✓'].shape[0]
    print(f"\nModels achieving >98.5% accuracy: {models_above_target}/{len(results)}")
    
    # Save comparison table
    comparison_df.to_csv('model_comparison_sequential.csv', index=False)
    print("\n✓ Comparison table saved to 'model_comparison_sequential.csv'")
else:
    print("\n⚠️  No models were trained. Please run at least one model training cell above.")

In [ ]:
# Visualize comparison
import matplotlib.pyplot as plt
import seaborn as sns

if len(results) > 0:
    # Set style
    sns.set_style('whitegrid')
    plt.figure(figsize=(12, 6))
    
    # Plot accuracy comparison
    plt.subplot(1, 2, 1)
    colors = ['green' if x == '✓' else 'red' for x in comparison_df['Meets Target (>98.5%)']]
    plt.barh(comparison_df['Model'], comparison_df['Accuracy (%)'], color=colors, alpha=0.7)
    plt.axvline(x=98.5, color='blue', linestyle='--', label='Target (98.5%)')
    plt.xlabel('Accuracy (%)')
    plt.title('Model Accuracy Comparison')
    plt.legend()
    plt.tight_layout()
    
    # Plot metrics comparison
    plt.subplot(1, 2, 2)
    metrics_df = comparison_df[['Model', 'Precision (%)', 'Recall (%)', 'F1-Score (%)']].set_index('Model')
    metrics_df.plot(kind='bar', ax=plt.gca(), width=0.8)
    plt.ylabel('Score (%)')
    plt.title('Model Metrics Comparison')
    plt.xticks(rotation=45, ha='right')
    plt.legend(loc='lower right')
    plt.tight_layout()
    
    plt.savefig('model_comparison_visualization.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✓ Visualization saved to 'model_comparison_visualization.png'")
else:
    print("⚠️  No results to visualize. Train at least one model first.")

In [ ]:
# Generate detailed classification reports
from sklearn.metrics import classification_report

print("="*80)
print("DETAILED CLASSIFICATION REPORTS")
print("="*80)

for model_name in all_models:
    try:
        y_pred = np.load(f"{model_name}_y_pred.npy")
        y_true = np.load(f"{model_name}_y_true.npy")
        
        y_pred_labels = y_pred.argmax(axis=1)
        y_true_labels = y_true.argmax(axis=1)
        
        print(f"\n{'='*80}")
        print(f"{model_name} - Classification Report")
        print(f"{'='*80}")
        
        report = classification_report(y_true_labels, y_pred_labels, target_names=class_names)
        print(report)
        
        # Save report
        with open(f"{model_name}_classification_report.txt", 'w') as f:
            f.write(f"{model_name} - Classification Report\n")
            f.write("="*80 + "\n")
            f.write(report)
        
        print(f"✓ Report saved to '{model_name}_classification_report.txt'")
    except FileNotFoundError:
        print(f"\n⚠️  {model_name} was not trained (skipped)")
    except Exception as e:
        print(f"Warning: Could not generate report for {model_name}: {str(e)}")

In [ ]:
# Final summary
print("\n" + "="*80)
print("FINAL SUMMARY")
print("="*80)

if len(results) > 0:
    print(f"\nTotal models trained: {len(results)}")
    print(f"Models achieving >98.5% accuracy: {models_above_target}")
    print(f"\nBest performing model: {comparison_df.iloc[0]['Model']}")
    print(f"Best accuracy: {comparison_df.iloc[0]['Accuracy (%)']:.2f}%")
else:
    print("\n⚠️  No models were trained.")

print("\nKey features implemented:")
print("  ✓ Sequential training (one model at a time)")
print("  ✓ Separate cells for each model (skip any model)")
print("  ✓ Dataset caching for faster training")
print("  ✓ Memory management between models")
print("  ✓ GPU memory growth enabled")
print("  ✓ Mixed precision training")
print("  ✓ Two-stage training with fine-tuning")
print("  ✓ Enhanced data augmentation")
print("  ✓ Learning rate scheduling")
print("  ✓ Early stopping and model checkpointing")
print("\n" + "="*80)
print("TRAINING COMPLETE!")
print("="*80)